In [6]:
import numpy as np
from Bio import SeqIO
import tables
#from bert_serving.client import BertClient
from ProduceKmerSequence import SeqGenerator,KmerSeqIterable,SeqMapper,SeqFragmenter,SlidingKmerFragmenter
import tokenization 
celline = "IMR90"
file_pre  = 'aug_50/'+celline
rand_seed=7
print(file_pre)
vocab_file='vocab.txt'
tokenizer = tokenization.FullTokenizer(vocab_file)

aug_50/IMR90


In [7]:
vocab_file='vocab.txt'
tokenizer = tokenization.FullTokenizer(vocab_file)
#output_tokens=tokenizer.tokenize('atgttt tgtt gttta' )
#input_ids=tokenizer.convert_tokens_to_ids(output_tokens)
#print(input_ids)

In [8]:
def GetEnhancerIndex(efile):
    seq_index_e=[]
    enhancer_sequences = SeqIO.parse(open(efile), 'fasta')
    rng=np.random.RandomState(rand_seed)
    kmer_fragmenter = SlidingKmerFragmenter(3, 8)
    for enhancer in enhancer_sequences:
        name, sequence = enhancer.id, str(enhancer.seq)
        #print(sequence)
        kmer_seq_iterable = KmerSeqIterable(
        rand_seed,
        SeqGenerator(sequence),
        SeqMapper(),
        SeqFragmenter(),
        kmer_fragmenter
    )
        kmer_seq_list=next(kmer_seq_iterable.__iter__())
        #print(len(kmer_seq_list))
        kmer_seq_str=' '.join(kmer_seq_list)
        output_tokens=tokenizer.tokenize(kmer_seq_str)
        input_ids=tokenizer.convert_tokens_to_ids(output_tokens)
        if len(input_ids)==2993:
            seq_index_e.append(input_ids)
        else:
            print(len(input_ids),name)
    X_e = np.array(seq_index_e)
    print(X_e.shape)
    return X_e
        
        
def GetPromoterIndex(pfile):
    seq_index_p=[]
    promoter_sequences = SeqIO.parse(open(pfile), 'fasta')
    rng=np.random.RandomState(rand_seed)
    kmer_fragmenter = SlidingKmerFragmenter(3, 8)
    for promoter in promoter_sequences:
        name, sequence = promoter.id, promoter.seq
        kmer_seq_iterable = KmerSeqIterable(
        rand_seed,
        SeqGenerator(sequence),
        SeqMapper(),
        SeqFragmenter(),
        kmer_fragmenter
    )
        kmer_seq_list=next(kmer_seq_iterable.__iter__())
        kmer_seq_str=' '.join(kmer_seq_list)
        output_tokens=tokenizer.tokenize(kmer_seq_str)
        input_ids=tokenizer.convert_tokens_to_ids(output_tokens)
        #promoter_index=str(input_ids).replace(r",","").replace(r"[","").replace(r"]","")
        if len(input_ids)==1993:
            seq_index_p.append(input_ids)
        else:
            print(len(input_ids),name)
                
    #print(seq_index_p)
    X_p = np.array(seq_index_p)
    print(X_p.shape)
    
    return X_p


In [5]:
#X_e=GetEnhancerIndex(efile = file_pre+'_enhancer.fasta')
#X_p=GetPromoterIndex(pfile = file_pre+'_promoter.fasta')

In [6]:
def GetTrainToken(labelfile):
    num_pos = 0
    num_neg = 0    
    with open(labelfile) as f:
        for line in f:
            if line.rstrip() == '1':
                num_pos = num_pos+1
            else:
                num_neg = num_neg+1
    label = np.concatenate([np.ones([num_pos,1]), np.zeros([num_neg,1])])
    X_e=GetEnhancerIndex(efile = file_pre+'_enhancer.fasta')
    X_p=GetPromoterIndex(pfile = file_pre+'_promoter.fasta')
    return X_e[0:num_pos],X_e[num_pos:], X_p[0:num_pos], X_p[num_pos:],label
    #yield X_e[0:num_pos],X_e[num_pos:], X_p[0:num_pos], X_p[num_pos:],label



In [7]:
seq_ep, seq_en,seq_pp, seq_pn,label=GetTrainToken(labelfile = file_pre+'_label.txt')
np.savez(celline+'_train.npz',enhancer_pos=seq_ep,enhancer_neg=seq_en,promoter_pos=seq_pp,promoter_neg=seq_pn,label=label)

(45080, 2993)
(45080, 1993)


In [12]:
def GetTestToken(labelfile):
    num_pos = 0
    num_neg = 0    
    with open(labelfile) as f:
        for line in f:
            if line.rstrip() == '1':
                num_pos = num_pos+1
            else:
                num_neg = num_neg+1
    label = np.concatenate([np.ones([num_pos,1]), np.zeros([num_neg,1])])
    X_e=GetEnhancerIndex(efile = file_pre+'_enhancer_test.fasta')
    X_p=GetPromoterIndex(pfile = file_pre+'_promoter_test.fasta')
    return X_e[0:num_pos],X_e[num_pos:], X_p[0:num_pos], X_p[num_pos:],label
    #yield X_e[0:num_pos],X_e[num_pos:], X_p[0:num_pos], X_p[num_pos:],label


In [13]:
seq_ep, seq_en,seq_pp, seq_pn,label=GetTestToken(labelfile = file_pre+'_label_test.txt')
np.savez(celline+'_test.npz',enhancer_pos=seq_ep,enhancer_neg=seq_en,promoter_pos=seq_pp,promoter_neg=seq_pn,label=label)

(2625, 2993)
(2625, 1993)
